In [1]:
import os
from groq import Groq
import pandas as pd
import numpy as np

In [2]:
apiKey = "fill_it_here"

In [3]:
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY"),
    api_key= apiKey,
)

In [4]:
dataset = pd.read_csv("keyphrases/done/keyphrases_NetflixMixtral_Done.csv")

In [5]:
dataset

,Unnamed: 0,content,keyphrases_original,cluster,sentiment,count,keyphrases_reduced,Merged keyphrases_reduced,count merged keyphrases_reduced,kw newly generated,id Mkr,Merged keyphrases_reduced 2,count 2,kw newly generated 2,count 2 off,id Mkr 2
0,0,Thank you for giving me the chance to build up...,"['credit rating', 'credit score', 'credit limi...",53,2,11,"['Amazon App User Experience', 'Credit Score I...","['Amazon App User Experience', 'Credit Score I...",11,NaN,0,"['Amazon App User Experience', 'Credit Score I...",11.0,NaN,11,0
1,51,Not easy but hopefully I've done it 😞,"['Amazon App', 'Difficulty', 'Feedback', 'expe...",53,2,11,"['Amazon App User Experience', 'Credit Score I...","['Amazon App User Experience', 'Credit Score I...",11,NaN,0,"['Amazon App User Experience', 'Credit Score I...",11.0,NaN,11,0
2,135,Wish shipping was faster for my area but I lov...,"['Amazon App', 'Shipping Speed', 'Wish Faster'...",53,1,11,"['Amazon App User Experience', 'Credit Score I...","['Amazon App User Experience', 'Credit Score I...",11,NaN,0,"['Amazon App User Experience', 'Credit Score I...",11.0,NaN,11,0
3,436,"I'm just trying it out, so hopefully it's good","['amazon app', 'first impression', 'testing', ...",53,2,11,"['Amazon App User Experience', 'Credit Score I...","['Amazon App User Experience', 'Credit Score I...",11,NaN,0,"['Amazon App User Experience', 'Credit Score I...",11.0,NaN,11,0
4,592,I guess it nice but i haven't updated it yet,"['Amazon App', 'UPDATE', 'NICE', 'USERS', 'COM...",53,2,11,"['Amazon App User Experience', 'Credit Score I...","['Amazon App User Experience', 'Credit Score I...",11,NaN,0,"['Amazon App User Experience', 'Credit Score I...",11.0,NaN,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,2558,Where is the dark mode? It's 2022! I have a Ga...,"['Amazon app', 'dark mode', 'Galaxy 21', 'Andr...",455,1,1,"['Amazon app', 'dark mode', 'Galaxy 21', 'Andr...","['Amazon app', 'dark mode', 'Galaxy 21', 'Andr...",1,NaN,426,"['Amazon app', 'dark mode', 'Galaxy 21', 'Andr...",1.0,NaN,1,378
2696,2596,This is well aswome and don't let the haters g...,"['Amazon App', 'Good Luck', ""Don't Let Haters ...",223,2,1,"['Amazon App', 'Good Luck', ""Don't Let Haters ...","['Amazon App', 'Good Luck', ""Don't Let Haters ...",1,NaN,427,"['Amazon app', 'good app', 'bad app', 'custome...",4.0,True,3,373
2697,2648,This version is heavier than before. Stripe so...,"['app size', 'update', 'loading speed', 'perfo...",229,1,1,"['app size', 'update', 'loading speed', 'perfo...","['app size', 'update', 'loading speed', 'perfo...",1,NaN,428,"['Amazon app issues', 'app problems', 'perform...",5.0,True,8,354
2698,2650,Don't use this company. Literally everything I...,"['poor quality', 'bad experience', 'warranty i...",458,0,1,"['poor quality', 'bad experience', 'warranty i...","['poor quality', 'bad experience', 'warranty i...",1,NaN,429,"['Amazon app', 'feedback issues', 'bad experie...",4.0,True,4,364


In [6]:
score_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(score_counts_balanced)

sentiment
0    900
1    900
2    900
Name: count, dtype: int64


In [7]:
train_data = dataset[['content', 'sentiment', 'Merged keyphrases_reduced 2']]
train_data['row_number'] = train_data.index

/tmp/ipykernel_1736366/1289041381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['row_number'] = train_data.index


In [8]:
train_data

,content,sentiment,Merged keyphrases_reduced 2,row_number
0,Thank you for giving me the chance to build up...,2,"['Amazon App User Experience', 'Credit Score I...",0
1,Not easy but hopefully I've done it 😞,2,"['Amazon App User Experience', 'Credit Score I...",1
2,Wish shipping was faster for my area but I lov...,1,"['Amazon App User Experience', 'Credit Score I...",2
3,"I'm just trying it out, so hopefully it's good",2,"['Amazon App User Experience', 'Credit Score I...",3
4,I guess it nice but i haven't updated it yet,2,"['Amazon App User Experience', 'Credit Score I...",4
...,...,...,...,...
2695,Where is the dark mode? It's 2022! I have a Ga...,1,"['Amazon app', 'dark mode', 'Galaxy 21', 'Andr...",2695
2696,This is well aswome and don't let the haters g...,2,"['Amazon app', 'good app', 'bad app', 'custome...",2696
2697,This version is heavier than before. Stripe so...,1,"['Amazon app issues', 'app problems', 'perform...",2697
2698,Don't use this company. Literally everything I...,0,"['Amazon app', 'feedback issues', 'bad experie...",2698


In [ ]:
list = []
i = 0
for index, row in train_data.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - negative, 2 - neutral, or 3 - positive. Here are examples to guide your classification:

                1. Comment: "so boring to use on android tv box. player is too obsolete with no menu and good look and feel. you can only play and pause. please upgrade your video player to have all options."
                Topics / Keyphrases: ['Netflix app issues', 'Video quality problems', 'Playback issues', 'App behavior changes', 'Notifications and user settings', 'Microphone permission and usage', 'HDR and display settings', 'Download quality options', 'Volume control and interface', 'Subscription and pricing concerns']
                Label: 1
                
                2. Comment: "Expensive and you can't share your account with your family."
                Topics / Keyphrases: ['Netflix price', 'expensive streaming service', 'worth the cost', 'high cost of subscription', 'issue with account sharing', 'consequences of password sharing', 'problem with password sharing', 'easier wallet management', 'too expensive for family sharing', 'improve user experience']
                Label: 1
                
                3. Comment: "Since I updated the app on my phone it's not worked and keeps saying cannot reach server, and doesn't work or load up."
                Topics / Keyphrases: ['Netflix app error', 'Update issue', 'Server connection problem', 'App not working', 'Watch list not loading', 'Profile access issue', 'Login issue', 'Device compatibility problems', 'Streaming issue', 'Technical difficulty']
                Label: 1                
                
                4. Comment: "Please don't waste your mobile data and storage. Do not download Netflix."
                Topics / Keyphrases: ['Netflix account limitations', 'Data waste due to streaming', 'Multi-user support for Netflix', 'Streaming concerns on mobile devices', 'Netflix storage management', 'Netflix download limitations', 'Mobile data conservation for Netflix', 'Netflix password sharing issues', 'App criticisms and limitations', 'Streaming service constraints on devices']
                Label: 1
                
                5. Comment: "App wouldn't let me watch anything or Evan load the main menu sfter finishing the sign up"
                Topics / Keyphrases: ['Netflix app loading error', 'app open issue', 'launch failure', 'error accessing', 'error message APP', 'app crashing problem', 'installation failed issue', 'technical problem with installation', 'loading main menu problem', 'app malfunction error']
                Label: 1
                
                6. Comment: "Good overall but definitely room for improvement especially in relation to multitasking. The Netflix app totally cuts out any ability to multitask this app alongside another. 👎👎 This is includes normal multitasking on android devices and also does not work as a multi window app in Samsung DeX. This is extremely frustrating as it is something lots of people would like to be able to do is multitask and watch your favourite shows or movies at the same time on Netflix. Please update this."
                Topics / Keyphrases: ['Netflix app issues', 'Video quality problems', 'Playback issues', 'App behavior changes', 'Notifications and user settings', 'Microphone permission and usage', 'HDR and display settings', 'Download quality options', 'Volume control and interface', 'Subscription and pricing concerns']
                Label: 2
                
                7. Comment: "Good but no password sharing and limits downloads with the plan you have 🥺"
                Topics / Keyphrases: ['Netflix account limitations', 'Data waste due to streaming', 'Multi-user support for Netflix', 'Streaming concerns on mobile devices', 'Netflix storage management', 'Netflix download limitations', 'Mobile data conservation for Netflix', 'Netflix password sharing issues', 'App criticisms and limitations', 'Streaming service constraints on devices']
                Label: 2
                                
                8. Comment: "You can mostly watch everything if it lets you"
                Topics / Keyphrases: ['Netflix app subscription', 'Watch free BTD6 on Netflix', 'Netflix subscription payment', 'Watch movies on Netflix', 'Netflix streaming service feature', 'Require payment for Netflix', 'Viewing options on Netflix app', 'Stream movies without downloading Netflix', 'Resume button on Netflix app', 'Easy Netflix app user interface']
                Label: 2
                                                
                9. Comment: "The app is good but most of their series they only post season 1 only or 2 instead of full series so one is left hanging without the rest of the series"
                Topics / Keyphrases: ['Netflix app performance issues', 'Series and movie streaming problems', 'Content delay and cancellation', 'App errors and crashes', 'Stream quality and buffering issues', 'Availability and selection of content', 'Download and offline watch issues', 'User experience and satisfaction', 'Technical glitches and bugs', 'Streaming service reliability and stability']
                Label: 2                
                
                10. Comment: "I love netfilx, but please add The Conjuring All parts, Annabelle all parts and The nun all parts in saudi arabia netfilx."
                Topics / Keyphrases: ['Netflix app content problem', 'Add or remove episodes', 'Missing TV shows and movies', 'Request content suggestion', 'TV show and movie availability', 'Content upload and rating', 'App feature request', 'Stream and watch problem', 'Subscription and payment issue', 'User voice and feedback']
                Label: 2

                11. Comment: "Incredible, fun and a lot of movies to explore"
                Topics / Keyphrases: ['Positive Netflix Experience', 'Streaming Service', 'Entertainment Content', 'Movie and Show Selection', 'Exploration and Browsing', 'Fun and Streamed Movies', 'High Quality Content', 'Good Movies to Watch', 'Netflix App Features', 'Streaming and Viewing Experience']
                Label: 3

                12. Comment: "It's a very nice app that allows you to watch whatever you wanna watch just by pressing the resume button"
                Topics / Keyphrases: ['Netflix app subscription', 'Watch free BTD6 on Netflix', 'Netflix subscription payment', 'Watch movies on Netflix', 'Netflix streaming service feature', 'Require payment for Netflix', 'Viewing options on Netflix app', 'Stream movies without downloading Netflix', 'Resume button on Netflix app', 'Easy Netflix app user interface']
                Label: 3
                
                13. Comment: "This app is nice and it has nice movies but it sometimes takes out nice movies before we even watch them"
                Topics / Keyphrases: ['Netflix app performance issues', 'Series and movie streaming problems', 'Content delay and cancellation', 'App errors and crashes', 'Stream quality and buffering issues', 'Availability and selection of content', 'Download and offline watch issues', 'User experience and satisfaction', 'Technical glitches and bugs', 'Streaming service reliability and stability']
                Label: 3
                
                14. Comment: "It's so great great ìts amazing it unbelievable its....I'm out of adjectives. Well it's great enyway 100 and 100000 %worth down Loading"
                Topics / Keyphrases: ['Netflix App', 'Movie Streaming', '5 Star Rating', 'User Review', 'Positive Experience', 'Loading Issue', 'App Performance', 'Content Availability', 'Favorite App', 'Lagging Problem']
                Label: 3
                
                15. Comment: "Great movie selections, easy of navigation thur the app ability to operate on phone, tablet or desktop."
                Topics / Keyphrases: ['Netflix app navigation', 'Easy user experience', 'Movie and TV show selection', 'Streamlined content interface', 'Friendly user interface', 'Good entertainment experience', 'Menu options and settings', 'Video quality issues', 'Netflix app performance on phone', 'Netflix app usability guidelines']
                Label: 3

                
                Please provide the sentiment classification as a single number (1, 2, or 3) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [10]:
len(list)

2700

In [ ]:
rest = train_data[len(list):]
for index, row in rest.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - negative, 2 - neutral, or 3 - positive. Here are examples to guide your classification:

                1. Comment: "so boring to use on android tv box. player is too obsolete with no menu and good look and feel. you can only play and pause. please upgrade your video player to have all options."
                Topics / Keyphrases: ['Netflix app issues', 'Video quality problems', 'Playback issues', 'App behavior changes', 'Notifications and user settings', 'Microphone permission and usage', 'HDR and display settings', 'Download quality options', 'Volume control and interface', 'Subscription and pricing concerns']
                Label: 1
                
                2. Comment: "Expensive and you can't share your account with your family."
                Topics / Keyphrases: ['Netflix price', 'expensive streaming service', 'worth the cost', 'high cost of subscription', 'issue with account sharing', 'consequences of password sharing', 'problem with password sharing', 'easier wallet management', 'too expensive for family sharing', 'improve user experience']
                Label: 1
                
                3. Comment: "Since I updated the app on my phone it's not worked and keeps saying cannot reach server, and doesn't work or load up."
                Topics / Keyphrases: ['Netflix app error', 'Update issue', 'Server connection problem', 'App not working', 'Watch list not loading', 'Profile access issue', 'Login issue', 'Device compatibility problems', 'Streaming issue', 'Technical difficulty']
                Label: 1                
                
                4. Comment: "Please don't waste your mobile data and storage. Do not download Netflix."
                Topics / Keyphrases: ['Netflix account limitations', 'Data waste due to streaming', 'Multi-user support for Netflix', 'Streaming concerns on mobile devices', 'Netflix storage management', 'Netflix download limitations', 'Mobile data conservation for Netflix', 'Netflix password sharing issues', 'App criticisms and limitations', 'Streaming service constraints on devices']
                Label: 1
                
                5. Comment: "App wouldn't let me watch anything or Evan load the main menu sfter finishing the sign up"
                Topics / Keyphrases: ['Netflix app loading error', 'app open issue', 'launch failure', 'error accessing', 'error message APP', 'app crashing problem', 'installation failed issue', 'technical problem with installation', 'loading main menu problem', 'app malfunction error']
                Label: 1
                
                6. Comment: "Good overall but definitely room for improvement especially in relation to multitasking. The Netflix app totally cuts out any ability to multitask this app alongside another. 👎👎 This is includes normal multitasking on android devices and also does not work as a multi window app in Samsung DeX. This is extremely frustrating as it is something lots of people would like to be able to do is multitask and watch your favourite shows or movies at the same time on Netflix. Please update this."
                Topics / Keyphrases: ['Netflix app issues', 'Video quality problems', 'Playback issues', 'App behavior changes', 'Notifications and user settings', 'Microphone permission and usage', 'HDR and display settings', 'Download quality options', 'Volume control and interface', 'Subscription and pricing concerns']
                Label: 2
                
                7. Comment: "Good but no password sharing and limits downloads with the plan you have 🥺"
                Topics / Keyphrases: ['Netflix account limitations', 'Data waste due to streaming', 'Multi-user support for Netflix', 'Streaming concerns on mobile devices', 'Netflix storage management', 'Netflix download limitations', 'Mobile data conservation for Netflix', 'Netflix password sharing issues', 'App criticisms and limitations', 'Streaming service constraints on devices']
                Label: 2
                                
                8. Comment: "You can mostly watch everything if it lets you"
                Topics / Keyphrases: ['Netflix app subscription', 'Watch free BTD6 on Netflix', 'Netflix subscription payment', 'Watch movies on Netflix', 'Netflix streaming service feature', 'Require payment for Netflix', 'Viewing options on Netflix app', 'Stream movies without downloading Netflix', 'Resume button on Netflix app', 'Easy Netflix app user interface']
                Label: 2
                                                
                9. Comment: "The app is good but most of their series they only post season 1 only or 2 instead of full series so one is left hanging without the rest of the series"
                Topics / Keyphrases: ['Netflix app performance issues', 'Series and movie streaming problems', 'Content delay and cancellation', 'App errors and crashes', 'Stream quality and buffering issues', 'Availability and selection of content', 'Download and offline watch issues', 'User experience and satisfaction', 'Technical glitches and bugs', 'Streaming service reliability and stability']
                Label: 2                
                
                10. Comment: "I love netfilx, but please add The Conjuring All parts, Annabelle all parts and The nun all parts in saudi arabia netfilx."
                Topics / Keyphrases: ['Netflix app content problem', 'Add or remove episodes', 'Missing TV shows and movies', 'Request content suggestion', 'TV show and movie availability', 'Content upload and rating', 'App feature request', 'Stream and watch problem', 'Subscription and payment issue', 'User voice and feedback']
                Label: 2

                11. Comment: "Incredible, fun and a lot of movies to explore"
                Topics / Keyphrases: ['Positive Netflix Experience', 'Streaming Service', 'Entertainment Content', 'Movie and Show Selection', 'Exploration and Browsing', 'Fun and Streamed Movies', 'High Quality Content', 'Good Movies to Watch', 'Netflix App Features', 'Streaming and Viewing Experience']
                Label: 3

                12. Comment: "It's a very nice app that allows you to watch whatever you wanna watch just by pressing the resume button"
                Topics / Keyphrases: ['Netflix app subscription', 'Watch free BTD6 on Netflix', 'Netflix subscription payment', 'Watch movies on Netflix', 'Netflix streaming service feature', 'Require payment for Netflix', 'Viewing options on Netflix app', 'Stream movies without downloading Netflix', 'Resume button on Netflix app', 'Easy Netflix app user interface']
                Label: 3
                
                13. Comment: "This app is nice and it has nice movies but it sometimes takes out nice movies before we even watch them"
                Topics / Keyphrases: ['Netflix app performance issues', 'Series and movie streaming problems', 'Content delay and cancellation', 'App errors and crashes', 'Stream quality and buffering issues', 'Availability and selection of content', 'Download and offline watch issues', 'User experience and satisfaction', 'Technical glitches and bugs', 'Streaming service reliability and stability']
                Label: 3
                
                14. Comment: "It's so great great ìts amazing it unbelievable its....I'm out of adjectives. Well it's great enyway 100 and 100000 %worth down Loading"
                Topics / Keyphrases: ['Netflix App', 'Movie Streaming', '5 Star Rating', 'User Review', 'Positive Experience', 'Loading Issue', 'App Performance', 'Content Availability', 'Favorite App', 'Lagging Problem']
                Label: 3
                
                15. Comment: "Great movie selections, easy of navigation thur the app ability to operate on phone, tablet or desktop."
                Topics / Keyphrases: ['Netflix app navigation', 'Easy user experience', 'Movie and TV show selection', 'Streamlined content interface', 'Friendly user interface', 'Good entertainment experience', 'Menu options and settings', 'Video quality issues', 'Netflix app performance on phone', 'Netflix app usability guidelines']
                Label: 3

                
                Please provide the sentiment classification as a single number (1, 2, or 3) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [21]:
len(list)

2700

In [12]:
# list.pop()

In [13]:
list

['Based on the provided examples, I classify the sentiment of the given comment as:\n\nLabel: 3\n\nThe comment expresses gratitude to Amazon for giving the opportunity to improve credit rating, which is a positive statement, indicating a positive sentiment.',
 'Based on the comment "Not easy but hopefully I\'ve done it 😞" and the provided topics/keyphrases, I would classify the sentiment as 2 (neutral). The comment expresses a sense of accomplishment, but the tone is more matter-of-fact than enthusiastic, and the 😞 emoji adds a hint of frustration or difficulty, which balances out the overall sentiment.',
 'The sentiment classification for this comment is:\n\nLabel: 2',
 'The sentiment classification for the given comment is:\n\nLabel: 2',
 'Label: 2',
 'Label: 3',
 'Label: 2',
 'I would classify the sentiment of the given comment as 3 - positive.',
 'The sentiment classification for this comment is:\n\n**2**',
 'Label: 3',
 'Based on the given examples and guidelines, I would categori

In [14]:
train_data['predictedSentiment'] = pd.Series(list)

/tmp/ipykernel_1736366/1772939642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['predictedSentiment'] = pd.Series(list)


In [15]:
train_data

,content,sentiment,Merged keyphrases_reduced 2,row_number,predictedSentiment
0,Thank you for giving me the chance to build up...,2,"['Amazon App User Experience', 'Credit Score I...",0,"Based on the provided examples, I classify the..."
1,Not easy but hopefully I've done it 😞,2,"['Amazon App User Experience', 'Credit Score I...",1,"Based on the comment ""Not easy but hopefully I..."
2,Wish shipping was faster for my area but I lov...,1,"['Amazon App User Experience', 'Credit Score I...",2,The sentiment classification for this comment ...
3,"I'm just trying it out, so hopefully it's good",2,"['Amazon App User Experience', 'Credit Score I...",3,The sentiment classification for the given com...
4,I guess it nice but i haven't updated it yet,2,"['Amazon App User Experience', 'Credit Score I...",4,Label: 2
...,...,...,...,...,...
2695,Where is the dark mode? It's 2022! I have a Ga...,1,"['Amazon app', 'dark mode', 'Galaxy 21', 'Andr...",2695,Label: 1
2696,This is well aswome and don't let the haters g...,2,"['Amazon app', 'good app', 'bad app', 'custome...",2696,"Based on the analysis, I would categorize the ..."
2697,This version is heavier than before. Stripe so...,1,"['Amazon app issues', 'app problems', 'perform...",2697,The sentiment classification for the given com...
2698,Don't use this company. Literally everything I...,0,"['Amazon app', 'feedback issues', 'bad experie...",2698,Label: 1


In [16]:
import re

def contains_number(s):
    return any(char.isdigit() for char in s)

def findNumbers(string):
    # Find all numbers in the string
    numbers = None
    if(contains_number(string)):
        numbers = re.findall(r'\d+', string)
    else:
        numbers = ""
    
    # Convert the numbers to integers
    
    numbers = [int(x) for x in numbers]
    
    if(len(numbers)>0):
        x = numbers[0]-1
    else:
        x = -1
    return x

In [17]:
formated = train_data['predictedSentiment'].apply(findNumbers)

In [18]:
train_data['predictedSentiment'] = formated

/tmp/ipykernel_1736366/188460712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['predictedSentiment'] = formated


In [19]:
train_data

,content,sentiment,Merged keyphrases_reduced 2,row_number,predictedSentiment
0,Thank you for giving me the chance to build up...,2,"['Amazon App User Experience', 'Credit Score I...",0,2
1,Not easy but hopefully I've done it 😞,2,"['Amazon App User Experience', 'Credit Score I...",1,1
2,Wish shipping was faster for my area but I lov...,1,"['Amazon App User Experience', 'Credit Score I...",2,1
3,"I'm just trying it out, so hopefully it's good",2,"['Amazon App User Experience', 'Credit Score I...",3,1
4,I guess it nice but i haven't updated it yet,2,"['Amazon App User Experience', 'Credit Score I...",4,1
...,...,...,...,...,...
2695,Where is the dark mode? It's 2022! I have a Ga...,1,"['Amazon app', 'dark mode', 'Galaxy 21', 'Andr...",2695,0
2696,This is well aswome and don't let the haters g...,2,"['Amazon app', 'good app', 'bad app', 'custome...",2696,2
2697,This version is heavier than before. Stripe so...,1,"['Amazon app issues', 'app problems', 'perform...",2697,1
2698,Don't use this company. Literally everything I...,0,"['Amazon app', 'feedback issues', 'bad experie...",2698,0


In [20]:
train_data.to_csv('mixtralOutput/topics/filenameNetflixFewShotMixtral-T.csv', index=False)

In [106]:
results = pd.read_csv("mixtralOutput/topics/filenameNetflixFewShotMixtral-T.csv")

In [ ]:
results

In [ ]:
results['predictedSentiment'].value_counts().sort_index()

In [109]:
filtered_df = results[results["predictedSentiment"] == -1]

In [ ]:
filtered_df

In [111]:
newResults = results[results["predictedSentiment"] != -1]

In [ ]:
newResults

In [113]:
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, classes, ymap=None, figsize=(17,17)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      classes:   aliases for the labels. String array to be shown in the cm plot.
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    sns.set(font_scale=2.8)

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            #elif c == 0:
            #    annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize='true')
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm = cm * 100
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    plt.yticks(va='center')

    sns.heatmap(cm, annot=annot, fmt='', ax=ax, xticklabels=classes, cbar=True, cbar_kws={'format':PercentFormatter()}, yticklabels=classes, cmap="Blues")
    plt.savefig(f'{filename}.png',  bbox_inches='tight')
    plt.savefig(f'{filename}.svg',  bbox_inches='tight')

In [114]:
# Generate the confusion matrix
unique_labels = np.sort(newResults['scoreSA'].unique())
mapping = {0: 'Very Negative (1)', 1: 'Negative (2)', 2: 'Neutral (3)', 3: 'Positive (4)', 4: 'Very Positive (5)'}

# Applying mapping
classes = [mapping[value] for value in unique_labels]

In [ ]:
cm_analysis(newResults['scoreSA'], newResults['predictedSentiment'], 'API', unique_labels, classes)

In [116]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, mean_absolute_error

accuracy = accuracy_score(newResults['scoreSA'], newResults['predictedSentiment'])
precision_ALL = precision_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
recall_ALL = recall_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
f1_ALL = f1_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
mae_ALL = mean_absolute_error(newResults['scoreSA'], newResults['predictedSentiment'])
precision, recall, f1, support = precision_recall_fscore_support(newResults['scoreSA'], newResults['predictedSentiment'])

In [ ]:
# Calculate accuracy for each class
for i in range(len(unique_labels)):
    temp = newResults[newResults["scoreSA"] == i]
    value = accuracy_score(temp['scoreSA'], temp['predictedSentiment'])
    print(f'Label:\t{i}\t-\tValue:\t{value}')

In [ ]:
accuracy

In [ ]:
balanced_accuracy = balanced_accuracy_score(results['scoreSA'], results['predictedSentiment'])

In [ ]:
balanced_accuracy

In [ ]:
precision_ALL

In [ ]:
recall_ALL

In [ ]:
f1_ALL

In [ ]:
mae_ALL

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1

In [ ]:
support